## **Citation Network Analysis**

This started as an adaptation of a [kaggle notebook](https://www.kaggle.com/code/virajjayant/citation-network-analysis), but has diverged significantly with the aim of creating a graph of the entire dataset rather than just 130 rows.

In [ ]:
import sqlite3
from pprint import pprint
import igraph as ig

import matplotlib.pyplot as plt

## Get Data
The dataset we will use is from [kaggle](https://www.kaggle.com/datasets/mathurinache/citation-network-dataset?resource=download). It comes zipped up so the first thing we want to do is extract it and examine the format.


In [ ]:
%%bash 

unzip data/dblp.v12.json.zip -d data

## Data Format
Using head and tail we can quickly get a sense for the content of the file without reading it all in.

In [ ]:
%%bash
# show first three lines, using cat to add line numbers
head data/dblp.v12.json -n 3 | cat -n 

In [ ]:
%%bash
# show last three lines, using cat to add line numbers
tail data/dblp.v12.json -n 3 | cat -n

In [ ]:
%%bash
# count the number of lines in the file
wc -l data/dblp.v12.json

So, the data probably looks like this:

```
[
{dict}
,{dict}
... ~ 4.9m more dicts
,{dict}
,{dict}
]
```

Looking at the above a few things are interesting:
* The dicts are each on their own line
* The commas separating the dicts are at the start of the line from the third line onwards

## ndjson 
This means the data is close to being in [ndjson](http://ndjson.org/) format where each line is a valid json object. This would be helpful because it would mean you can parse the file a line at a time, instead of having to swallow the whole thing.

We can massage the file into shape with some sed (only takes a few seconds)

In [ ]:
%%bash

sed -i '1d;$d' data/dblp.v12.json # remove 1st & last line
sed -i 's/^,//' data/dblp.v12.json # remove commas where they are the first character using regex
wc -l data/dblp.v12.json # see - two fewer rows.

# `sqlite-utils`
Now we are in ndjson form we can use [sqlite-utils](https://sqlite-utils.datasette.io) to create an sqlite database of the file, which will let us work with it without having to have it all it memory.

## Some Failed Attempts
My first attempt was to use sqlite & its json1 extension to create the table directly, but I think I need a slightly newer version of sqlite to get readfile working

```sql
create table citations_direct as SELECT json_extract(value, '$.id') from json_each(readfile('dblp.v12.json'));

```

The next thing I tried was sqlite-utils:
```sh
sqlite-utils insert test.db citations --pk=id dblp.v12.json --nl
> Error: table citations has no column named alias_ids
```


```sh
sqlite3 citations.db "CREATE TABLE test(id text, year number, title text, refs text)"
sqlite-utils bulk citations.db 'insert into test (id, year, title, refs) values (:id, :year, :title, :references)' dblp.v12.json --nl
> Error binding parameter :references - probably unsupported type.
```

## Brute Force
...alternatively we can just insert everything. This takes a minute, but works.

We are using `--nl` to tell it to expect newline delimited json & `--lines` to tell it to insert each line as an item in the line column of the citations table.

I have set the batch size between database commits to an arbitrary large number to increase performance (it defaults to 100) this could cause issues if we wanted to read the database while this process was running, but we dont so...

In [ ]:
%%bash
# 120s with large batch size vs ~460s without
sqlite-utils insert data/citations.db citations data/dblp.v12.json --nl --lines --batch-size 100000

## SQLite in Python
Here we make a connection to the database we just created

In [ ]:
conn = sqlite3.connect("data/citations.db")
cur = conn.cursor()

we can the use this to run SQL against the database. In this case, we check how many rows we have generated:

In [ ]:
cur.execute("SELECT COUNT(*) FROM citations")
cur.fetchone()

## SQLite & JSON

SQLite contains a set of functions for working with json, [see the docs](https://sqlite.org/json1.html).

We can use sqlite's `json_extract()` function to pull out the keys we need from the json

In [ ]:
cur.execute("""
CREATE TABLE subset AS 
    SELECT 
        json_extract(line, '$.id') as id,
        json_extract(line, '$.year') as yr, 
        json_extract(line, '$.title') as title,
        json_extract(line, '$.references') as refs
    FROM
        citations;
""")

We can check this has worked as intended by taking a look at the top of the table.

In [ ]:
cur.execute("SELECT * FROM subset LIMIT 3")
cur.fetchall()

### Unnest
That's great and all, but we have multiple items in the refs cells which we probably want to split out so we can deal with them sensibly.

Happily, the `json_tree()` function let you turn the elements of the json (or in this case the array in the refs column) into rows.

In [ ]:
cur.execute("""
SELECT 
    subset.id, atom
FROM 
    subset, json_tree(subset.refs) 
WHERE 
    json_tree.key IS NOT NULL  -- filters out the top level bits of the tree
LIMIT 10;                      -- we just want a preview
""")

cur.fetchall()

In this case, because we have pruned our json column down to just an array, we dont need the tree, we only need to traverse one level down, using `json_each()`.
So we can create a table of edges (i.e. paper id -> ref) with the following:

In [ ]:
cur.execute("""
CREATE TABLE edges AS 
    SELECT subset.id, atom
      FROM subset, json_each(subset.refs);
"""
)

In [ ]:
cur.execute("SELECT * FROM edges LIMIT 10")
cur.fetchall()

Neat - so now we have a the subset table with the information for our verticies and the edges table with just the citations info, with a row per citation.

# SQLite to igraph

Now we have used SQLite to do the data prep work we want to start constructing our graph. We will do this using the package `igraph` which provides bindings to the igraph C library which provides a performant graph data structure. (networkx is pretty slow).

To do this we need to bring our data out of sqlite and into python. (This is the bit where you start to use some RAM, but if we hit limits we can break this into chunks.)

In [ ]:
cur.execute("SELECT cast(id as text), cast(atom as text) FROM edges")
edges = cur.fetchall()
len(edges)

Thats 45,564,149 edges!

In [ ]:
edges[0:5]

Now the node / vertex information. In the first instance we probably just want the titles and paper id's.

We are casting to text here so that we can use them as node labels rather than node id's in igraph (integers get used as ids)

In [ ]:
cur.execute("SELECT cast(id as text), title FROM subset")
nodes = cur.fetchall()
len(nodes)

In [ ]:
nodes[:3]

At this point, checking memory usage we are using ~16Gb of RAM, so depending on the machine we may need to use some `LIMIT, OFFSET` queries to page through the results and work in smaller chunks.

In [ ]:
cgraph = ig.Graph(directed = True)
cgraph

The nodes come as a list of 4.9m tuples, but I want to flip the indicies and have a list of length two where the first item contains all the node labels, and the second the titles. I overwrite the original nodes variable to avoid eating even more RAM.

In [ ]:
nodes = list(zip(*nodes))

In [ ]:
nodes[0][:3]

In [ ]:
nodes[1][:3]

This makes it easy to pass the data into the bulk add_vertices & add_edges functions:

In [ ]:
cgraph.add_vertices(n = len(nodes[0]), attributes = {"name": nodes[0], "title":nodes[1]})

In [ ]:
cgraph.add_edges(edges)

In [ ]:
cgraph.vcount()

In [ ]:
cgraph.ecount()

RAM usage ~18Gb at this point

# Using the graph

I could imagine a situation where we wanted to search for a node in the graph, then find the other nodes it cites

In [ ]:
cur.execute("SELECT * FROM SUBSET WHERE title like '% igraph %' LIMIT 10")
search_results = cur.fetchall()
search_results

In [ ]:
str(search_results[0][0])

In [ ]:
# get the vertex with the name corresponding to the first search result.
vtx = cgraph.vs.find(name=str(search_results[0][0]))
vtx

In [ ]:
vtx.index

In [ ]:
neighbor_ids = cgraph.neighborhood(vtx.index, order=1)
neighbor_ids

In [ ]:
neighbor_vtx = ig.VertexSeq(cgraph, neighbor_ids)
neighbor_vtx

In [ ]:
neighbor_vtx["title"]

# Important Nodes

igraph gives us some options to start to sift through the graph and figure out which nodes are important.

## PageRank

See: https://en.wikipedia.org/wiki/PageRank


In [ ]:
pagerank = cgraph.pagerank()

In [ ]:
pagerank[0:10]

In [ ]:
len(pagerank)

In [ ]:
max(pagerank)

In [ ]:
import numpy as np

In [ ]:
# get index of the maximum pagerank
indexnp.argmax(pagerank)

In [ ]:
# get the vertex object for this index
max_pr_vertex = cgraph.vs[1651044]
max_pr_vertex

In [ ]:
# check its the right vertex by cross checking the pagerank
max_pr_vertex.pagerank()

In [ ]:
# see how many edges this vertex has
len(max_pr_vertex.all_edges())

In [ ]:
# of these, how many are edges going in (i.e. citing) the paper
len(max_pr_vertex.in_edges())

In [ ]:
# and how many are going out (i.e. what did it cite)
len(max_pr_vertex.out_edges())

In [ ]:
# yep - checks out, 966 rows in edges citing this paper.
cur.execute("SELECT count(*) FROM edges WHERE atom = 2054801208")
cur.fetchall()

Yeah, we can find this paper on google scholar [here](https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=Finite+automata+and+their+decision+problems&btnG=)

It seems like a good candidate for a high page rank - it's cited by >2000 papers (I guess mostly outside our dataset) and it only cites 7 itself

# Disconnect


In [ ]:
cur.close()
conn.close()